In [1]:
!pwd
%config IPCompleter.greedy=True # コードを自動補完
import sys

sys.executable

/app/my_task/Bike_Shareing/InClass_work/run_pycaret_v2


'/usr/local/bin/python'

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
from tqdm import tqdm_notebook as tqdm

import pycaret
from pycaret.regression import *

In [3]:
data_dir = "../../data/add_feature_v9_InClass"
output_dir = "model/tmp_setup/casual"
os.makedirs(output_dir, exist_ok=True)

df_train = pd.read_csv(
    os.path.join(data_dir, "train.csv"),
    parse_dates=["datetime"],
)
df_train = df_train.drop(["casual", "registered"], axis=1)  # train setにしかない説明変数 削除
df_test = pd.read_csv(
    os.path.join(data_dir, "test.csv"),
    parse_dates=["datetime"],
)

print(df_train.info())
display(
    df_train.head().style.background_gradient(cmap="Pastel1")
)  # style.background_gradient でデータフレームに色付ける
display(df_train.describe().style.background_gradient(cmap="Pastel1"))

print(df_test.info())
display(df_test.head().style.background_gradient(cmap="Pastel1"))
display(df_test.describe().style.background_gradient(cmap="Pastel1"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13003 entries, 0 to 13002
Columns: 303 entries, datetime to count_log
dtypes: datetime64[ns](1), float64(252), int64(49), object(1)
memory usage: 30.1+ MB
None


/usr/local/lib/python3.7/site-packages/pandas/io/formats/style.py:1126: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/usr/local/lib/python3.7/site-packages/pandas/io/formats/style.py:1127: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Columns: 302 entries, datetime to hour_count_log_te
dtypes: datetime64[ns](1), float64(251), int64(49), object(1)
memory usage: 10.1+ MB
None


In [4]:
# 列削除
_features = [
    "casual_log",
    "datetime",
    "temp",
    "atemp",
    "humidity",
    "windspeed",
    "count_season_mid",
    "season",
    "holiday",
    "workingday",
    "weather",
    "year",
    "dayofweek",
    "hour",
    "hour_workingday_casual",
    "hour_workingday_holiday_casual_v1",
    "hour_workingday_holiday_casual_v2",
    "casual_log_184_day_shift",
    "casual_log_185_day_shift",
    "casual_log_186_day_shift",
    "casual_log_187_day_shift",
    "casual_log_188_day_shift",
    "casual_log_189_day_shift",
    "casual_log_190_day_shift",
    "casual_log_191_day_shift",
    "casual_log_192_day_shift",
    "casual_log_193_day_shift",
    "casual_log_194_day_shift",
    "casual_log_195_day_shift",
    "casual_log_196_day_shift",
    "casual_log_197_day_shift",
    "casual_log_198_day_shift",
    "casual_log_199_day_shift",
    "casual_log_200_day_shift",
    "casual_log_201_day_shift",
    "casual_log_202_day_shift",
    "casual_log_203_day_shift",
    "casual_log_dayofweek-hour_shift_1",
    "casual_log_dayofweek-hour_shift_10",
    "casual_log_dayofweek-hour_shift_11",
    "casual_log_dayofweek-hour_shift_12",
    "casual_log_dayofweek-hour_shift_13",
    "casual_log_dayofweek-hour_shift_14",
    "casual_log_dayofweek-hour_shift_15",
    "casual_log_dayofweek-hour_shift_16",
    "casual_log_dayofweek-hour_shift_17",
    "casual_log_dayofweek-hour_shift_18",
    "casual_log_dayofweek-hour_shift_19",
    "casual_log_dayofweek-hour_shift_2",
    "casual_log_dayofweek-hour_shift_20",
    "casual_log_dayofweek-hour_shift_21",
    "casual_log_dayofweek-hour_shift_22",
    "casual_log_dayofweek-hour_shift_23",
    "casual_log_dayofweek-hour_shift_24",
    "casual_log_dayofweek-hour_shift_25",
    "casual_log_dayofweek-hour_shift_26",
    "casual_log_dayofweek-hour_shift_3",
    "casual_log_dayofweek-hour_shift_4",
    "casual_log_dayofweek-hour_shift_5",
    "casual_log_dayofweek-hour_shift_6",
    "casual_log_dayofweek-hour_shift_7",
    "casual_log_dayofweek-hour_shift_8",
    "casual_log_dayofweek-hour_shift_9",
    "casual_log_shift_tail_mean",
    "season_casual_log_te",
    "year_casual_log_te",
    "month_casual_log_te",
    "dayofweek_casual_log_te",
    "hour_casual_log_te",
    "elapsed_day",
    "discomfort",
    "casual_log_cat_cols_mid",
    "holiday_casual_log_te",
    "workingday_casual_log_te",
]

df_train = df_train[_features]

_features.remove("casual_log")
df_test = df_test[_features]
df_test["casual_log"] = 0

print(df_train.shape, df_test.shape)
print(df_train.info())
print(df_test.info())

(13003, 74) (4376, 74)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13003 entries, 0 to 13002
Data columns (total 74 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   casual_log                          13003 non-null  float64       
 1   datetime                            13003 non-null  datetime64[ns]
 2   temp                                13003 non-null  float64       
 3   atemp                               13003 non-null  float64       
 4   humidity                            13003 non-null  float64       
 5   windspeed                           13003 non-null  float64       
 6   count_season_mid                    13003 non-null  int64         
 7   season                              13003 non-null  int64         
 8   holiday                             13003 non-null  int64         
 9   workingday                          13003 non-null  int64         
 10 

In [5]:
_session_id = 123
_target = "casual_log"
_silent = True
# _date_features = ["datetime"]
_ignore_features = ["datetime"]
model_name = "lightgbm"

# exp は (x, y, train_x, val_x, train_y, val_y, session_id, sklearn_pipeline)のタプル
exp = setup(
    df_train,
    session_id=_session_id,
    target=_target,
    silent=_silent,  # 「silent=True」を渡すことで、型推定の確認でenter押すのスキップ,
    # date_features=_date_features,
    # categorical_features=_categorical_features,
    # numeric_features=_numeric_features,
    ignore_features=_ignore_features,
    # bin_numeric_features=["windspeed"],  # ビン化したい数値データ特徴量のカラム名をリストで指定
    # normalize=True,  # 各numeric_featuresを正規化
    # normalize_method='zscore',  # 正規化の方法は標準化
    # normalize_method='minmax',  # Min-Maxスケーリング
    # remove_outliers=True,  # PCAを使用して外れ値除去
    # outliers_threshold=0.05,  # 外れ値の割合を指定。デフォルトの0.05を指定すると、分布の裾の両側の値の0.025%が除去
    # ignore_low_variance=True,  # 統計的に有意ではない分散を持つ特徴量を削除
    # polynomial_features=True,  # 全ての数値データ特徴量の多項式の組み合わせで新たな特徴量を生成
    # polynomial_degree=2,  # 多項式特徴の次数を指定。default = 2
    data_split_shuffle=False, # データシャッフルさせない
    folds_shuffle=False, # データシャッフルさせない
    #categorical_imputation='not_available',  # カテゴリ列の欠損置換しない
)

# 常に全ての列（カラム）を表示
pd.set_option("display.max_columns", None)
display(exp[0].head(3))
pd.concat([exp[0], exp[1]], axis=1).to_csv(
    os.path.join(output_dir, "train.csv"), index=False
)  # 一応前処理後のtrain set保存しておく

print(exp[0].shape)
print(exp[1][:5])
print(exp[1].apply(np.expm1)[:5])  # np.log1pの逆変換

 
Setup Succesfully Completed.


,Description,Value
0,session_id,123
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(13003, 74)"
4,Missing Values,True
5,Numeric Features,62
6,Categorical Features,10
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


,temp,atemp,humidity,windspeed,hour,casual_log_184_day_shift,casual_log_185_day_shift,casual_log_186_day_shift,casual_log_187_day_shift,casual_log_188_day_shift,casual_log_189_day_shift,casual_log_190_day_shift,casual_log_191_day_shift,casual_log_192_day_shift,casual_log_193_day_shift,casual_log_194_day_shift,casual_log_195_day_shift,casual_log_196_day_shift,casual_log_197_day_shift,casual_log_198_day_shift,casual_log_199_day_shift,casual_log_200_day_shift,casual_log_201_day_shift,casual_log_202_day_shift,casual_log_203_day_shift,casual_log_dayofweek-hour_shift_1,casual_log_dayofweek-hour_shift_10,casual_log_dayofweek-hour_shift_11,casual_log_dayofweek-hour_shift_12,casual_log_dayofweek-hour_shift_13,casual_log_dayofweek-hour_shift_14,casual_log_dayofweek-hour_shift_15,casual_log_dayofweek-hour_shift_16,casual_log_dayofweek-hour_shift_17,casual_log_dayofweek-hour_shift_18,casual_log_dayofweek-hour_shift_19,casual_log_dayofweek-hour_shift_2,casual_log_dayofweek-hour_shift_20,casual_log_dayofweek-hour_shift_21,casual_log_dayofweek-hour_shift_22,casual_log_dayofweek-hour_shift_23,casual_log_dayofweek-hour_shift_24,casual_log_dayofweek-hour_shift_25,casual_log_dayofweek-hour_shift_26,casual_log_dayofweek-hour_shift_3,casual_log_dayofweek-hour_shift_4,casual_log_dayofweek-hour_shift_5,casual_log_dayofweek-hour_shift_6,casual_log_dayofweek-hour_shift_7,casual_log_dayofweek-hour_shift_8,casual_log_dayofweek-hour_shift_9,casual_log_shift_tail_mean,season_casual_log_te,year_casual_log_te,month_casual_log_te,dayofweek_casual_log_te,hour_casual_log_te,elapsed_day,discomfort,casual_log_cat_cols_mid,holiday_casual_log_te,workingday_casual_log_te,count_season_mid_127,count_season_mid_165,count_season_mid_169,count_season_mid_77,season_1,season_2,season_3,season_4,holiday_0,holiday_1,workingday_0,workingday_1,weather_1,weather_2,weather_3,weather_4,year_2011,year_2012,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,hour_workingday_casual_0,hour_workingday_casual_1,hour_workingday_holiday_casual_v1_0,hour_workingday_holiday_casual_v1_1,hour_workingday_holiday_casual_v2_0,hour_workingday_holiday_casual_v2_1
0,0.24,0.2879,0.81,0.0,0.0,2.545561,2.548315,2.550772,2.554999,2.554862,2.556655,2.557482,2.560695,2.563678,2.569535,2.571361,2.572978,2.574388,2.574659,2.576188,2.579751,2.583677,2.5862,2.588976,2.589636,2.597126,2.511639,2.500469,2.491333,2.477391,2.468752,2.454383,2.443368,2.444637,2.452998,2.456778,2.587067,2.470109,2.482924,2.492272,2.498512,2.517678,2.531142,2.543935,2.575931,2.56412,2.553274,2.541663,2.530357,2.51933,2.510059,2.728463,1.866510,2.555332,1.563435,3.060181,1.917388,0.0,46.380495,2.302585,2.614348,3.023435,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.22,0.2727,0.80,0.0,1.0,2.545561,2.548315,2.550772,2.554999,2.554862,2.556655,2.557482,2.560695,2.563678,2.569535,2.571361,2.572978,2.574388,2.574659,2.576188,2.579751,2.583677,2.5862,2.588976,2.589636,2.597126,2.511639,2.500469,2.491333,2.477391,2.468752,2.454383,2.443368,2.444637,2.452998,2.456778,2.587067,2.470109,2.482924,2.492272,2.498512,2.517678,2.531142,2.543935,2.575931,2.56412,2.553274,2.541663,2.530357,2.51933,2.510059,2.728463,1.866510,2.555332,1.563435,3.060181,1.521728,0.0,46.365542,2.302585,2.614348,3.023435,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.22,0.2727,0.80,0.0,2.0,2.545561,2.548315,2.550772,2.554999,2.554862,2.556655,2.557482,2.560695,2.563678,2.569535,2.571361,2.572978,2.574388,2.574659,2.576188,2.579751,2.583677,2.5862,2.588976,2.589636,2.597126,2.511639,2.500469,2.491333,2.477391,2.468752,2.454383,2.443368,2.444637,2.452998,2.456778,2.587067,2.470109,2.482924,2.492272,2.498512,2.517678,2.531142,2.543935,2.575931,2.56412,2.553274,2.541663,2.530357,2.51933,2.510059,2.728463,1.848943,2.546483,1.535723,3.065737,1.195873,0.0,46.365542,2.302585,2.608834,3

(13003, 93)
0    1.386294
1    2.197225
2    1.791759
3    1.386294
4    0.000000
Name: casual_log, dtype: float64
0    3.0
1    8.0
2    5.0
3    3.0
4    0.0
Name: casual_log, dtype: float64


In [6]:
_session_id = 123
_target = "casual_log"
_silent = True
# _date_features = ["datetime"]
_ignore_features = ["datetime"]
model_name = "lightgbm"

# exp は (x, y, train_x, val_x, train_y, val_y, session_id, sklearn_pipeline)のタプル
exp = setup(
    df_test,
    session_id=_session_id,
    target=_target,
    silent=_silent,  # 「silent=True」を渡すことで、型推定の確認でenter押すのスキップ,
    # date_features=_date_features,
    # categorical_features=_categorical_features,
    # numeric_features=_numeric_features,
    ignore_features=_ignore_features,
    # bin_numeric_features=["windspeed"],  # ビン化したい数値データ特徴量のカラム名をリストで指定
    # normalize=True,  # 各numeric_featuresを正規化
    # normalize_method='zscore',  # 正規化の方法は標準化
    # normalize_method='minmax',  # Min-Maxスケーリング
    # remove_outliers=True,  # PCAを使用して外れ値除去
    # outliers_threshold=0.05,  # 外れ値の割合を指定。デフォルトの0.05を指定すると、分布の裾の両側の値の0.025%が除去
    # ignore_low_variance=True,  # 統計的に有意ではない分散を持つ特徴量を削除
    # polynomial_features=True,  # 全ての数値データ特徴量の多項式の組み合わせで新たな特徴量を生成
    # polynomial_degree=2,  # 多項式特徴の次数を指定。default = 2
    data_split_shuffle=False, # データシャッフルさせない
    folds_shuffle=False, # データシャッフルさせない
    #categorical_imputation='not_available',  # カテゴリ列の欠損置換しない
)

# 常に全ての列（カラム）を表示
pd.set_option("display.max_columns", None)
display(exp[0].head(3))
pd.concat([exp[0], exp[1]], axis=1).to_csv(
    os.path.join(output_dir, "test.csv"), index=False
)  # 一応前処理後のtrain set保存しておく

print(exp[0].shape)
print(exp[1][:5])
print(exp[1].apply(np.expm1)[:5])  # np.log1pの逆変換

 
Setup Succesfully Completed.


,Description,Value
0,session_id,123
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(4376, 74)"
4,Missing Values,True
5,Numeric Features,60
6,Categorical Features,12
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


,temp,atemp,humidity,windspeed,hour,casual_log_184_day_shift,casual_log_185_day_shift,casual_log_186_day_shift,casual_log_187_day_shift,casual_log_188_day_shift,casual_log_189_day_shift,casual_log_190_day_shift,casual_log_191_day_shift,casual_log_192_day_shift,casual_log_193_day_shift,casual_log_194_day_shift,casual_log_195_day_shift,casual_log_196_day_shift,casual_log_197_day_shift,casual_log_198_day_shift,casual_log_199_day_shift,casual_log_200_day_shift,casual_log_201_day_shift,casual_log_202_day_shift,casual_log_203_day_shift,casual_log_dayofweek-hour_shift_1,casual_log_dayofweek-hour_shift_10,casual_log_dayofweek-hour_shift_11,casual_log_dayofweek-hour_shift_12,casual_log_dayofweek-hour_shift_13,casual_log_dayofweek-hour_shift_14,casual_log_dayofweek-hour_shift_15,casual_log_dayofweek-hour_shift_16,casual_log_dayofweek-hour_shift_17,casual_log_dayofweek-hour_shift_18,casual_log_dayofweek-hour_shift_19,casual_log_dayofweek-hour_shift_2,casual_log_dayofweek-hour_shift_20,casual_log_dayofweek-hour_shift_21,casual_log_dayofweek-hour_shift_22,casual_log_dayofweek-hour_shift_23,casual_log_dayofweek-hour_shift_24,casual_log_dayofweek-hour_shift_25,casual_log_dayofweek-hour_shift_26,casual_log_dayofweek-hour_shift_3,casual_log_dayofweek-hour_shift_4,casual_log_dayofweek-hour_shift_5,casual_log_dayofweek-hour_shift_6,casual_log_dayofweek-hour_shift_7,casual_log_dayofweek-hour_shift_8,casual_log_dayofweek-hour_shift_9,casual_log_shift_tail_mean,season_casual_log_te,year_casual_log_te,month_casual_log_te,dayofweek_casual_log_te,hour_casual_log_te,elapsed_day,discomfort,casual_log_cat_cols_mid,count_season_mid_127,count_season_mid_169,count_season_mid_77,season_1,season_3,season_4,holiday_0,holiday_1,workingday_0,workingday_1,weather_1,weather_2,weather_3,year_2012,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,hour_workingday_casual_0,hour_workingday_casual_1,hour_workingday_holiday_casual_v1_0,hour_workingday_holiday_casual_v1_1,hour_workingday_holiday_casual_v2_0,hour_workingday_holiday_casual_v2_1,holiday_casual_log_te_2.603695236337777,holiday_casual_log_te_2.7433128557906885,workingday_casual_log_te_2.4179759290876532,workingday_casual_log_te_3.0192815261402868
0,0.76,0.7273,0.66,0.0000,0.0,1.609438,1.609438,0.0,1.386294,1.791759,1.098612,1.386294,1.098612,1.386294,2.079442,0.000000,1.386294,2.079442,1.386294,2.564949,1.791759,0.0,0.693147,1.609438,1.791759,3.465736,3.496508,3.433987,3.091042,2.197225,2.944439,3.332205,2.079442,1.791759,2.079442,2.302585,3.465736,0.000000,1.386294,2.772589,0.693147,2.302585,2.708050,1.791759,3.526361,3.332205,3.891820,3.761200,2.995732,3.178054,0.000000,3.661872,3.170042,2.739358,3.337178,3.053369,1.902969,547.0,46.826186,4.219508,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.74,0.6970,0.70,0.1343,1.0,2.302585,0.000000,0.0,0.000000,1.386294,1.098612,0.693147,0.693147,0.000000,0.693147,0.000000,1.386294,1.386294,1.609438,1.386294,1.791759,0.0,0.000000,0.000000,2.197225,3.688879,2.397895,2.890372,2.708050,2.639057,3.218876,3.258097,1.609438,2.833213,1.791759,2.397895,3.091042,0.000000,1.791759,1.791759,0.693147,1.098612,2.397895,2.772589,3.496508,3.091042,2.890372,3.367296,3.135494,3.637586,1.791759,3.623984,3.170042,2.739358,3.337178,3.053369,1.495546,547.0,46.804428,4.219508,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.72,0.6970,0.74,0.0896,2.0,1.098612,0.000000,0.0,1.098612,1.098612,0.000000,1.609438,0.693147,0.693147,0.000000,0.693147,0.000000,1.098612,2.484907,0.000000,1.098612,0.0,0.693147,0.000000,1.791759,3.555348,2.079442,3.044522,2.079442,3.044522,1.945910,2.772589,2.708050,1.791759,1.386294,0.693147,2.995732,1.609438,1.386294,0.693147,1.791759,2.397895,2.833213,0.000000,2.890372,3.135494,2.833213,3.610918,3.135494,3.401197,1.609438,3.566774,3.170042,2.739358,3.337178,3.053369,1.205109,54

(4376, 91)
0    0
1    0
2    0
3    0
4    0
Name: casual_log, dtype: int64
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: casual_log, dtype: float64
